See instructions here:

https://github.com/tapilab/elevate-osna-starter/blob/master/lessons/week2/README.md#day-2

In [2]:
from collections import Counter
import numpy as np
import pandas as pd
import re
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [150]:
def load_data(datafile, checkfile):
    """
    Read your data into a single pandas dataframe where
    - each row is an instance to be classified
    
    (this could be a tweet, user, or news article, depending on your project)
    - there is a column called `label` which stores the class label (e.g., the true
      category for this row)
    """
    df = pd.read_csv(datafile)[['social_id','comment_tokens','comment_time']]
    ck = pd.read_csv(checkfile)
    
    ck = ck.loc[ck['site'] == 'twitter', ['site', 'social_id', 'ruling_val']]
    
    
    ck['social_id'] = ck['social_id'].astype(df['social_id'].dtype)
    
    df.columns = ['id', 'text','time']
#     df = df.drop_duplicates(['id','text'])
    ck.columns = ['site','id','label']
    df = pd.merge(ck,df,on=['id'],how = 'inner')
    df['label'] = ['true' if i>0 else 'false' if i<0 else 'unknown' for i in df.label]
    df['comments_count'] = 1
    df['timemin'] = df['time']
    
    # combine multiple rows of an id into one row
    def ab(df):
        return' '.join(df.values)
    df = df.groupby(['id','label']).agg({'text':ab,'comments_count':sum,'time':max,'timemin':min})
    df['timeslot'] = df['time']-df['timemin']
    df['timepercomm'] = df['timeslot']/df['comments_count']
    df['timepercomm'] = [int(i) for i in df.timepercomm]
    df = df.drop(['time','timemin'],axis=1)
    df = df.reset_index()
    
    return df

df = load_data('..\\..\\training_data\\twitter.csv', '..\\..\\training_data\\factchecks.csv')
df.head()

,id,label,text,comments_count,timeslot,timepercomm
0,1972425520,true,rt @senjohnmccain : obama ha more czar than th...,7,78173527,11167646
1,2554608773,false,rt @thatkevinsmith : ten year in and we bone l...,87,1384707,15916
2,10288400197,true,rt @jimdemint : house appropriation chair davi...,13,63956,4919
3,15561382074,false,rt @markos : i get one for politifactref - - i...,2,73,36
4,16415781807,unknown,rt @presssec : who would the gop put in charge...,84,3116561,37101


In [110]:
# what is the distribution over class labels?
df.label.value_counts()

false      793
true       224
unknown    197
Name: label, dtype: int64

In [111]:
def make_features(df):
    vec = DictVectorizer()
    feature_dicts = []
    # just as an initial example, we will consider three
    # word features in the model.
    words_to_track = ['you', 'hate', 'love']
    # will get different model for different features.
    for i, row in df.iterrows():
        features = {}
        token_counts = Counter(re.sub('\W+', ' ', row['text'].lower()).split())
        for w in words_to_track:
            features[w] = token_counts[w]
        feature_dicts.append(features)
    X = vec.fit_transform(feature_dicts)
    return X, vec
                
X, vec = make_features(df)

In [112]:
X
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html

<1214x3 sparse matrix of type '<class 'numpy.float64'>'
	with 101 stored elements in Compressed Sparse Row format>

In [113]:
# Now, use CountVectorizer to create a term feature matrix.
count_vec = CountVectorizer()
X_words = count_vec.fit_transform(df.text)
X_words.shape

(1214, 5519)

In [114]:
# how sparse is it? 
def print_sparsity(X_words):
    print('%d words' % X_words.shape[1])
    num_cells = X_words.shape[0] * X_words.shape[1]
    print('%d of %d possible cells are non-zero (%.2f%%)' %
          (X_words.nnz, num_cells,
           100 * X_words.nnz/num_cells))
print_sparsity(X_words)

5519 words
21101 of 6700066 possible cells are non-zero (0.31%)


In [115]:
# how does sparsity vary with min_df?
for min_df in [1,2,5,10]:
    count_vec = CountVectorizer(min_df=min_df)
    print('\n\nmin_df=%d' % min_df)
    X_words = count_vec.fit_transform(df.text)
    print_sparsity(X_words)



min_df=1
5519 words
21101 of 6700066 possible cells are non-zero (0.31%)


min_df=2
1953 words
17535 of 2370942 possible cells are non-zero (0.74%)


min_df=5
653 words
14198 of 792742 possible cells are non-zero (1.79%)


min_df=10
289 words
11850 of 350846 possible cells are non-zero (3.38%)


In [116]:
# top terms?
def print_top_words(X_words, count_vec, n=10):
    features = count_vec.get_feature_names()
    word_counts = X_words.sum(axis=0).A1
    for i in np.argsort(word_counts)[::-1][:n]:
        print('%20s\t%d' % (features[i], word_counts[i]))

print_top_words(X_words, count_vec)

                  rt	69813
                 the	43717
              urlref	36591
                  be	35204
                  to	31350
                  in	20232
                  of	19614
                 and	15533
     realdonaldtrump	14273
                 for	10573


In [117]:
# top terms using different ngrams
for ngram in [(1,1), (2,2), (3,3), (1,3)]:
    count_vec = CountVectorizer(min_df=2, ngram_range=ngram)
    print('\nngram_range=%s' % str(ngram))
    X_words = count_vec.fit_transform(df.text)
    print_sparsity(X_words)
    print_top_words(X_words, count_vec)


ngram_range=(1, 1)
1953 words
17535 of 2370942 possible cells are non-zero (0.74%)
                  rt	69813
                 the	43717
              urlref	36591
                  be	35204
                  to	31350
                  in	20232
                  of	19614
                 and	15533
     realdonaldtrump	14273
                 for	10573

ngram_range=(2, 2)
1814 words
6224 of 2202196 possible cells are non-zero (0.28%)
           urlref rt	30885
  rt realdonaldtrump	11569
              of the	3610
       urlref urlref	2826
              in the	2775
             this be	2591
              to the	2540
              on the	2158
               to be	1986
              be the	1745

ngram_range=(3, 3)
516 words
1304 of 626424 possible cells are non-zero (0.21%)
    urlref urlref rt	2771
urlref rt realdonaldtrump	1358
urlref rt hillaryclinton	1023
rt realdonaldtrump the	949
     today urlref rt	587
         there be no	492
          one of the	479
      this urlref rt	474
      

In [118]:
# we'll first store the classes separately in a numpy array
y = np.array(df.label)
Counter(y)

Counter({'false': 793, 'true': 224, 'unknown': 197})

In [120]:
# store the class names
class_names = set(df.label)

In [121]:
# how often does each word appear in each class?
for word, idx in list(vec.vocabulary_.items())[:10]:
    for class_name in class_names:
        class_idx = np.where(y==class_name)[0]
        print('%20s\t%20s\t%d' % (word, class_name, X[class_idx, idx].sum()))

                 you	               false	3616
                 you	             unknown	1352
                 you	                true	1341
                hate	               false	352
                hate	             unknown	0
                hate	                true	87
                love	               false	500
                love	             unknown	0
                love	                true	85


In [122]:
# to combine two sparse matrices:
from scipy.sparse import csr_matrix, hstack # "horizontal stack"
# m1 = csr_matrix([[1,2,3], [4,5,6]])
# print('m1')
# print(m1.todense())
# m2 = csr_matrix([[7,8], [9,10]])
# print('m2')
# print(m2.todense())
# hstack([m1, m2]).todense()

In [123]:
X_all = hstack([X, X_words])
X_all.shape

(1214, 4286)

In [124]:
# fit a LogisticRegression classifier.
clf = LogisticRegression(solver='lbfgs', multi_class='multinomial')
clf.fit(X_all, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [125]:
# for binary classification, LogisticRegression stores a single coefficient vector
clf.coef_
# this would be a matrix for a multi-class probem.

array([[ 0.03080239,  0.06097997, -0.01932483, ..., -0.00907405,
         0.02402858,  0.02402858],
       [ 0.00434927,  0.01893742,  0.02934549, ..., -0.02399279,
        -0.01215877, -0.01215877],
       [-0.03515166, -0.07991739, -0.01002066, ...,  0.03306685,
        -0.01186981, -0.01186981]])

In [126]:
clf.coef_.shape

(3, 4286)

In [127]:
# for binary classification, the coefficients for the negative class is just the negative of the positive class.
coef = clf.coef_
print(coef)

[[ 0.03080239  0.06097997 -0.01932483 ... -0.00907405  0.02402858
   0.02402858]
 [ 0.00434927  0.01893742  0.02934549 ... -0.02399279 -0.01215877
  -0.01215877]
 [-0.03515166 -0.07991739 -0.01002066 ...  0.03306685 -0.01186981
  -0.01186981]]


In [128]:
clf.classes_

array(['false', 'true', 'unknown'], dtype=object)

In [129]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

kf = KFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []
for train, test in kf.split(X):
    clf.fit(X[train], y[train])
    pred = clf.predict(X[test])
    accuracies.append(accuracy_score(y[test], pred))
    
    
print('accuracy over all cross-validation folds: %s' % str(accuracies))
print('mean=%.2f std=%.2f' % (np.mean(accuracies), np.std(accuracies)))

accuracy over all cross-validation folds: [0.6337448559670782, 0.6748971193415638, 0.6296296296296297, 0.6584362139917695, 0.6570247933884298]
mean=0.65 std=0.02
